In [1]:
import re
puzzle = '''Dancer can fly 27 km/s for 5 seconds, but then must rest for 132 seconds.
Cupid can fly 22 km/s for 2 seconds, but then must rest for 41 seconds.
Rudolph can fly 11 km/s for 5 seconds, but then must rest for 48 seconds.
Donner can fly 28 km/s for 5 seconds, but then must rest for 134 seconds.
Dasher can fly 4 km/s for 16 seconds, but then must rest for 55 seconds.
Blitzen can fly 14 km/s for 3 seconds, but then must rest for 38 seconds.
Prancer can fly 3 km/s for 21 seconds, but then must rest for 40 seconds.
Comet can fly 18 km/s for 6 seconds, but then must rest for 103 seconds.
Vixen can fly 18 km/s for 5 seconds, but then must rest for 84 seconds.'''
time = 2503

In [2]:
matcher = re.compile(r'(?P<name>[A-Za-z]+?) can fly (?P<speed>\d+) km/s for (?P<active>\d+) seconds, but then must rest for (?P<rest>\d+) seconds\.')
reindeer = {}
for line in puzzle.splitlines():
    match = matcher.match(line).groupdict()
    name = match['name']
    match['speed'], match['active'], match['rest'] = int(match['speed']), int(match['active']), int(match['rest'])
    reindeer[name] = match

In [3]:
def distance(reindeer, time):
    cycle_distance = reindeer['active']*reindeer['speed']
    cycle_time = reindeer['active'] + reindeer['rest']
    cycles = time // cycle_time
    leftover = time % cycle_time
    total = cycle_distance * cycles
    if leftover <= reindeer['active']:
        total += leftover*reindeer['speed']
    else:
        total += cycle_distance
    return total

In [4]:
for deer in reindeer.values():
    deer['total'] = distance(deer, time)
max(deer['total'] for deer in reindeer.values())

2640

In [5]:
for seconds in range(1, time+1):
    for deer in reindeer.values():
        deer['total'] = distance(deer, seconds)
    lead = max(deer['total'] for deer in reindeer.values())
    leaders = [deer for deer in reindeer.values() if deer['total'] == lead]
    for leader in leaders:
        leader['points'] = leader.get('points', 0) + 1
max(deer.get('points', 0) for deer in reindeer.values())

1102